In [1]:
import pandas as pd
import numpy as np

In [2]:
df_all = []

In [ ]:
# original R4W TABLE
def get_volume_r4w(df, start_wk, end_wk):
    rolling_window_df = pd.DataFrame([
    {
        'rolling_order':i,
        'rolling_start': end - pd.Timedelta(weeks=4),
        'roling_end':end,
        'rolling_window_range_start': (end - pd.Timedelta(weeks=4)).strftime('%Y-%m-%d') + ' to ' + end.strftime('%Y-%m-%d'),
        'rollingw_window_range_end': end.strftime('%Y-%m-%d') + ' to ' + (end + pd.Timedelta(weeks=4)).strftime('%Y-%m-%d')
    }
    for i, end in enumerate(pd.date_range(start=start_wk, end = end_wk, freq='W-FRI'))])

    #step1 build all accounts x rolling_order combinations
    all_accounts = df_all['account_id'].unique()
    all_orders = rolling_window_df['rolling_order'].unique()
    account_order_grid = pd.MultiIndex.from_product([all_accounts, all_orders], 
                                                    names=['Account ID', 'rolling_order']).to_frame(index=False)
    
    #stept2: compute metrics only where data exists
    volume_records = []
    for acct in df_all['account_id'].unique():
        df_acct = df_all[df_all['account_id'] == acct]
        for _, window in rolling_window_df.iterrows():
            df_window = df_acct[(df_acc['week'] > window['rolling_start']) & (df_acc['week'] <= window['roling_end'])]
            if df_window.empty:
                continue
            a_stock = df_window['a_stock'].sum()
            b_stock = df_window['b_stock'].sum()
            c_stock = df_window['c_stock'].sum()
            total = a_stock + b_stock + c_stock
            a_freq = (df_window['a_stock'] > 0).sum() / df_window['week'].nunique()
            b_freq = (df_window['b_stock'] > 0).sum() / df_window['week'].nunique()
            #c_freq = (df_window['c_stock'] > 0).sum() / df_window['week'].nunique()
            ms_a = a_stock/total if total > 0 else None
            ms_b = b_stock/total if total > 0 else None
            volume_records.append({
                'Account ID': acct,
                'rolling_order': window['rolling_order'],
                'rolling_4w_a_stock': a_stock,
                'rolling_4w_b_stock': b_stock,
                'c_stock': c_stock,
                #'total_stock': total,
                'a_freq': a_freq,
                'b_freq': b_freq,
                #'c_freq': c_freq,
                'rolling_4w_a_ms': ms_a,
                'rolling_4w_a_ms': ms_b,
            })
    volume_partial = pd.DataFrame(volume_records)
    volume_r4w_full = account_order_grid.merge(volume_partial, on=['Account ID', 'rolling_order'], how='left')

    volume_r4w = volume_r4w_full.merge(rolling_window_df, on='rolling_order', how='left')
    metrics_cols = ['rolling_4w_a_stock', 'rolling_4w_b_stock', 'c_stock', 'a_freq', 'b_freq', 
                    'rolling_4w_a_ms', 'rolling_4w_a_ms']
    volume_r4w[metrics_cols] = volume_r4w[metrics_cols].fillna(0)
    volume_r4w = volume_r4w.sort_values(['Account ID', 'rolling_order'])
    volume_r4w = volume_r4w.merge(df_all[['account_id', 'account_name']].drop_duplicates(),
                                left_on='Account ID', right_on='account_id', how='left').drop(columns=['account_id'])
    print(df_all['account_id'].nunique(), volume_r4w['Account ID'].nunique(), len(volume_r4w.groupby(['Account ID'])['rolling_order'].nunique().unique().min()))
    return volume_r4w
    
    

In [ ]:
# spcale up version:
import pandas as pd
import numpy as np

# ✅ 建议：开启 pandas 2.x Copy-on-Write（若版本支持），减少不必要拷贝
# Recommended: enable pandas 2.x copy-on-write (if your version supports it)
pd.options.mode.copy_on_write = True


# def build_r4w_metrics(
#     df: pd.DataFrame,
#     start_wk: str | pd.Timestamp,
#     end_wk: str | pd.Timestamp,
#     *,
#     week_col: str = "week",
#     id_col: str = "account_id",
#     name_col: str = "account_name",
#     a_col: str = "a_stock",
#     b_col: str = "b_stock",
#     c_col: str = "c_stock",
#     freq_denominator: str = "fixed4"  # "fixed4"（默认，分母恒等于4）或 "observed"（分母=窗口内实际观测周数）
# ) -> pd.DataFrame:
    
def build_r4w_metrics(
    df,
    start_wk,
    end_wk,
    *,
    week_col="week",
    id_col="account_id",
    name_col="account_name",
    a_col="a_stock",
    b_col="b_stock",
    c_col="c_stock",
    freq_denominator="fixed4"
) -> pd.DataFrame:

    """
    生成以“周五”为窗口终点的 R4W（滚动4周）指标（纯 pandas 向量化、高性能）
    Build R4W (rolling 4 weeks) metrics with Friday as window end (pure pandas, vectorized & fast)

    输入 Input
    -----
    df : 包含原始周度数据的 DataFrame；需包含：
         DataFrame with weekly rows; must contain:
         - id_col（门店/账户ID；e.g., "account_id"）
         - name_col（门店名；e.g., "account_name"）
         - week_col（周日期；可为YYYYMMDD/字符串/日期；会对齐到该周“周五”）
         - a_col / b_col（A/B品类销量）
         - c_col（可选；若不存在将按0处理）
    start_wk, end_wk : 统计区间（窗口终点的起/止周五）；如 "2025-01-03", "2025-10-03"
                       Friday range for window ends, e.g. "2025-01-03" to "2025-10-03"
    freq_denominator : "fixed4" 使用固定分母4（推荐，稳定）；"observed" 使用窗口内实际观测周数作为分母
                       Use "fixed4" (recommended) or "observed" for frequency denominator.

    输出 Output
    -----
    返回含以下列的 DataFrame（只保留窗口终点位于 [start_wk, end_wk] 的周五）：
    Returns a DataFrame with Friday window ends within [start_wk, end_wk], including:
      - {id_col}, {name_col}, {week_col}（窗口终点 Friday / window end Friday）
      - rolling_order（窗口序号，按时间1..N / sequential order 1..N）
      - rolling_start, rolling_end（窗口起止；(end-4周, end] / window bounds）
      - rolling_window_range_start / rolling_window_range_end（可读范围 / human-readable ranges）
      - rolling_4w_a_stock / rolling_4w_b_stock / rolling_4w_c_stock（近4周各品类总量 / 4-week sums）
      - total_stock（近4周总量 / 4-week total）
      - a_freq / b_freq / c_freq（近4周“>0的周”占比 / share of positive weeks in last 4）
      - rolling_4w_a_ms / rolling_4w_b_ms（近4周份额 / market share）
    """

    # ---------- 0) 选择所需列 & 统一周日期到“周五” ----------
    # Keep necessary columns only; unify week to the Friday of each week
    base_cols = [id_col, name_col, week_col, a_col, b_col]
    if c_col in df.columns:
        base_cols.append(c_col)
    df = df[base_cols].copy()

    # 将各种 week 表示（YYYYMMDD/字符串/日期）统一为 Timestamp，并对齐到该周周五
    # Normalize 'week' to Timestamp and align to Friday of the week (W-FRI)
    if not np.issubdtype(df[week_col].dtype, np.datetime64):
        s = df[week_col].astype(str)
        is_ymd = s.str.len().eq(8) & s.str.match(r"^\d{8}$")
        df.loc[is_ymd, week_col] = pd.to_datetime(s[is_ymd], format="%Y%m%d", errors="coerce")
        df.loc[~is_ymd, week_col] = pd.to_datetime(s[~is_ymd], errors="coerce")
    df[week_col] = df[week_col].dt.to_period("W-FRI").dt.to_timestamp("W-FRI")

    # ---------- 1) 缺失列与类型压缩 ----------
    # Fill missing c_col with 0 if absent; compress numeric dtypes
    if c_col not in df.columns:
        df[c_col] = 0

    for col in (a_col, b_col, c_col):
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype("Int32")

    # ---------- 2) 生成窗口终点（周五）序列 ----------
    # Build list of Friday window ends within [start_wk, end_wk]
    start_wk = pd.to_datetime(start_wk)
    end_wk = pd.to_datetime(end_wk)
    friday_ends = pd.date_range(start=start_wk, end=end_wk, freq="W-FRI")

    # 由于 rolling(4) 需要往前3周的数据，扩展一个最早日期 earliest_needed
    # For rolling(4), we need 3 more prior weeks
    earliest_needed = friday_ends.min() - pd.Timedelta(weeks=3)

    # ---------- 3) 聚合为“账号×周五”的周表，并对每个账号补齐完整周历 ----------
    # Aggregate to weekly (Friday) per account, then reindex each account to a full Friday calendar
    weekly = (
        df.groupby([id_col, name_col, week_col], as_index=False)[[a_col, b_col, c_col]].sum()
    )

    # def _reindex_one(g: pd.DataFrame) -> pd.DataFrame:
    #     idx = pd.date_range(start=earliest_needed, end=end_wk, freq="W-FRI")
    #     g = g.set_index(week_col).reindex(idx, fill_value=0)
    #     g.index.name = week_col
    #     g = g.reset_index()
    #     g[id_col] = g[id_col].iloc[0]
    #     g[name_col] = g[name_col].iloc[0]
    #     return g[[id_col, name_col, week_col, a_col, b_col, c_col]]
    def _reindex_one(g: pd.DataFrame) -> pd.DataFrame:
    # 先从真实行取出 id/name 常量，避免被补齐行的 0 覆盖
        _id   = g[id_col].iloc[0]
        _name = g[name_col].iloc[0]

        idx = pd.date_range(start=earliest_needed, end=end_wk, freq="W-FRI")

    # 只对数值列做补齐；不要对 id/name 做 fill_value=0
        g_vals = (
            g[[week_col, a_col, b_col, c_col]]
            .set_index(week_col)
            .reindex(idx, fill_value=0)
            .reset_index()
            .rename(columns={"index": week_col})
    )

    # 回写 id/name 常量
        g_vals[id_col] = _id
        g_vals[name_col] = _name

        return g_vals[[id_col, name_col, week_col, a_col, b_col, c_col]]

    weekly_full = (
        weekly.groupby([id_col, name_col], group_keys=False)
              .apply(_reindex_one)
              .reset_index(drop=True)
              .sort_values([id_col, week_col])
    )

    # ---------- 4) 计算 R4W 的 sum / freq / ms（全向量化） ----------
    # Compute rolling-4-week sums, positive frequencies, and market shares (vectorized)
    gb = weekly_full.groupby(id_col, group_keys=False)

    # 近4周 sum（A/B/C）
    a_4 = gb[a_col].rolling(4, min_periods=1).sum().reset_index(drop=True)
    b_4 = gb[b_col].rolling(4, min_periods=1).sum().reset_index(drop=True)
    c_4 = gb[c_col].rolling(4, min_periods=1).sum().reset_index(drop=True)

    weekly_full["rolling_4w_a_stock"] = a_4.astype("float")
    weekly_full["rolling_4w_b_stock"] = b_4.astype("float")
    weekly_full["rolling_4w_c_stock"] = c_4.astype("float")
    weekly_full["total_stock"] = weekly_full["rolling_4w_a_stock"] + weekly_full["rolling_4w_b_stock"] + weekly_full["rolling_4w_c_stock"]

    # 近4周“有货周”计数（>0）/ count of positive weeks in the last 4
    a_pos = gb[a_col].apply(lambda s: (s > 0).rolling(4, min_periods=1).sum()).reset_index(drop=True)
    b_pos = gb[b_col].apply(lambda s: (s > 0).rolling(4, min_periods=1).sum()).reset_index(drop=True)
    c_pos = gb[c_col].apply(lambda s: (s > 0).rolling(4, min_periods=1).sum()).reset_index(drop=True)

    # 频率分母：fixed4 = 4；observed = 窗口内实际观测周数（首个窗口可能<4）
    # Frequency denominator: fixed4=4; observed=actual observed weeks in the window (first windows may be <4)
    if freq_denominator == "observed":
        denom = (gb[week_col].cumcount() + 1).clip(upper=4)
    else:
        denom = pd.Series(4, index=weekly_full.index)

    weekly_full["a_freq"] = (a_pos / denom).astype("float")
    weekly_full["b_freq"] = (b_pos / denom).astype("float")
    weekly_full["c_freq"] = (c_pos / denom).astype("float")

    # 近4周份额（market share）；total=0 时置为 0，避免 NaN/inf
    # 4-week market shares; if total==0, set to 0.0
    with np.errstate(divide="ignore", invalid="ignore"):
        weekly_full["rolling_4w_a_ms"] = np.where(
            weekly_full["total_stock"] > 0,
            weekly_full["rolling_4w_a_stock"] / weekly_full["total_stock"],
            0.0,
        )
        weekly_full["rolling_4w_b_ms"] = np.where(
            weekly_full["total_stock"] > 0,
            weekly_full["rolling_4w_b_stock"] / weekly_full["total_stock"],
            0.0,
        )

    # ---------- 5) 仅保留窗口终点位于 [start_wk, end_wk] 的周五，并生成 rolling_order ----------
    # Keep only Friday ends within [start_wk, end_wk] and add rolling_order
    out = weekly_full[weekly_full[week_col].isin(friday_ends)].copy()
    out = out.sort_values([id_col, week_col])

    ro_map = {d: i + 1 for i, d in enumerate(sorted(friday_ends))}
    out["rolling_order"] = out[week_col].map(ro_map).astype("Int32")

    # 窗口边界与范围（(end-4周, end]）
    # Window bounds and human-readable ranges
    out["rolling_end"] = out[week_col]
    out["rolling_start"] = out["rolling_end"] - pd.Timedelta(weeks=4)
    out["rolling_window_range_start"] = (
        out["rolling_start"].dt.strftime("%Y-%m-%d") + " to " + out["rolling_end"].dt.strftime("%Y-%m-%d")
    )
    out["rolling_window_range_end"] = (
        out["rolling_end"].dt.strftime("%Y-%m-%d") + " to " + (out["rolling_end"] + pd.Timedelta(weeks=4)).dt.strftime("%Y-%m-%d")
    )

    # 输出列顺序 / Final column order
    keep_cols = [
        id_col, name_col, week_col, "rolling_order",
        "rolling_start", "rolling_end",
        "rolling_window_range_start", "rolling_window_range_end",
        "rolling_4w_a_stock", "rolling_4w_b_stock", "rolling_4w_c_stock",
        "total_stock", "a_freq", "b_freq", "c_freq",
        "rolling_4w_a_ms", "rolling_4w_b_ms",
    ]
    out = out.loc[:, keep_cols]

    return out


In [ ]:
volume_r4w = build_r4w_metrics(
    df_all,
    start_wk="2024-05-01",   # 不必是周五；函数会自动对齐到该周“周五”
    end_wk="2025-08-29",     # 这是周五，完美
    freq_denominator="fixed4"  # ✅ 正确参数名
)

In [ ]:
# Original Change Table Version
def get_change_r4w(volume_r4w):
    change_records = []
    volume_r4w['rolling_window_range'] = (
        volume_r4w['rolling_start'].dt.strftime('%Y-%m-%d') + 
        ' to ' 
        + volume_r4w['rolling_end'].dt.strftime('%Y-%m-%d')
    )

    for acct, df_acct in volume_r4w.groupby('Account ID'):
        df_acct = df_acct.sort_values('rolling_order').reset_index(drop=True)
        for i in range(1, len(df_acct)):
            prev = df_acct.iloc[i-1]
            curr = df_acct.iloc[i]

            def pct(now, prev): return round((now - prev)/prev, 2) if pd.notnull(prev) and prev != 0 else 0

            change_records.append({
                'Account ID': acct,
                'rolling_order': curr['rolling_order'],
                'rolling_window_range_start': prev['rolling_window_range'],
                'rolling_window_range_end': curr['rolling_window_range'],
                'change_4w_a_stock': pct(curr['rolling_4w_a_stock'], prev['rolling_4w_a_stock']),
                'change_4w_b_stock': pct(curr['rolling_4w_b_stock'], prev['rolling_4w_b_stock']),
                'change_c_stock': pct(curr['c_stock'] , prev['c_stock']),
                'change_a_freq': pct(curr['a_freq'] , prev['a_freq']),
                'change_b_freq': pct(curr['b_freq'] , prev['b_freq']),
                'change_4w_a_ms': pct(curr['rolling_4w_a_ms'] , prev['rolling_4w_a_ms']),
                'change_4w_b_ms': pct(curr['rolling_4w_b_ms'] , prev['rolling_4w_b_ms']),
            })
    change_r4w = pd.DataFrame(change_records)

    first_b_stocking_orders = (volume_r4w[(volume_r4w['rolling_4w_b_ms'] > 0)]
                               .groupby('Account ID')['rolling_order']
                               .min()
                               .reset_index()
                               .rename(columns={'rolling_order':'first_b_stock_rw'}))
    fallback_orders = (volume_r4w.groupby('Account ID')['rolling_order'].max().reset_index()
                          .rename(columns={'rolling_order':'fallback_order'}))
    first_b_stocking_orders = fallback_orders.merge(first_b_stocking_orders, on='Account ID', how='left')
    first_b_stocking_orders['first_b_stock_rw'] = first_b_stocking_orders['first_b_stock_rw'].fillna(first_b_stocking_orders['fallback_order'])
    change_r4w = change_r4w.merge(first_b_stocking_orders[['Account ID', 'first_b_stock_rw']],
                                  left_on='Account ID', right_on='Account ID', how='left')
    change_r4w = change_r4w.merge(df_all[['Account ID', 'account_name']].drop_duplicates(),
                                left_on='Account ID', right_on='Account ID', how='left')
    return change_r4w

In [ ]:
import pandas as pd
import numpy as np

def get_change_r4w_fast(
    volume_r4w: pd.DataFrame,
    *,
    id_col: str = "account_id",
    name_col: str = "account_name",
    order_col: str = "rolling_order",
    start_col: str = "rolling_start",
    end_col: str = "rolling_end",
    # R4W 指标列（与 build_r4w_metrics 输出保持一致）
    a4_col: str = "rolling_4w_a_stock",
    b4_col: str = "rolling_4w_b_stock",
    c4_col: str = "rolling_4w_c_stock",
    a_freq_col: str = "a_freq",
    b_freq_col: str = "b_freq",
    c_freq_col: str = "c_freq",
    a_ms_col: str = "rolling_4w_a_ms",
    b_ms_col: str = "rolling_4w_b_ms",
    # 是否保留绝对变化（delta）列
    keep_delta: bool = False,
    # 百分比保留小数位
    round_ndigits: int = 2,
):
    """
    生成“相邻滚动窗变化表”（相同账号、相邻 rolling_order 的百分比变化）。
    Vectorized change table between consecutive rolling windows per account.

    变化定义（默认）/ Default change definition:
        pct_change = (curr - prev) / prev
        若 prev==0 或 NaN 则记为 0（避免 inf/NaN）

    输出列 / Output columns:
      - id/name/order
      - rolling_window_range_start: 上一个窗口的 [start, end] 字符串
      - rolling_window_range_end  : 当前窗口的   [start, end] 字符串
      - change_4w_a_stock / change_4w_b_stock / change_4w_c_stock
      - change_a_freq / change_b_freq / change_c_freq
      - change_4w_a_ms / change_4w_b_ms
      - （可选）*_delta 绝对变化列
      - first_b_stock_rw: 每账号首次出现 B 份额>0 的 rolling_order（缺失回退为该账号最大 rolling_order）
    """

    df = volume_r4w.copy()

    # ---------- 0) 安全排序 / sort to ensure consecutive windows ----------
    df = df.sort_values([id_col, order_col], kind="mergesort").reset_index(drop=True)

    # ---------- 1) 生成窗口范围字符串（上、下两个窗） ----------
    # Build readable ranges first
    rng_str = (
        df[start_col].dt.strftime("%Y-%m-%d")
        + " to "
        + df[end_col].dt.strftime("%Y-%m-%d")
    )
    df["__range_str"] = rng_str

    # 上一个窗口（同账号，order-1）的范围与指标 → 使用 groupby().shift(1)
    # Previous window (same account) via shift(1)
    gb = df.groupby(id_col, group_keys=False)

    prev_range = gb["__range_str"].shift(1)
    prev_order = gb[order_col].shift(1)

    # 需要变化的字段列表（在这里统一维护，易扩展）
    metric_cols = [
        a4_col, b4_col, c4_col,
        a_freq_col, b_freq_col, c_freq_col,
        a_ms_col, b_ms_col,
    ]

    # 为每个度量生成 prev、delta、pct_change
    for col in metric_cols:
        df[f"__prev_{col}"]  = gb[col].shift(1)
        df[f"__delta_{col}"] = df[col] - df[f"__prev_{col}"]
        # 百分比变化： (curr-prev)/prev，prev==0 or NaN → 0
        prev = df[f"__prev_{col}"].astype("float")
        curr = df[col].astype("float")
        with np.errstate(divide="ignore", invalid="ignore"):
            pct = np.where((prev != 0) & (~np.isnan(prev)),
                           (curr - prev) / prev,
                           0.0)
        df[f"__pct_{col}"] = np.round(pct, round_ndigits)

    # 只保留具有“前一个窗口”的行（rolling_order >= 2）
    df_keep = df[prev_order.notna()].copy()

    # ---------- 2) 组装输出 ----------
    out = pd.DataFrame({
        id_col:   df_keep[id_col].values,
        name_col: df_keep[name_col].values,
        order_col: df_keep[order_col].astype("Int32").values,
        "rolling_window_range_start": prev_range.loc[df_keep.index].values,  # 上一窗口的范围
        "rolling_window_range_end":   df_keep["__range_str"].values,         # 当前窗口的范围
    })

    # 映射列名：原指标名 → 输出变化列名
    rename_map = {
        a4_col:      "change_4w_a_stock",
        b4_col:      "change_4w_b_stock",
        c4_col:      "change_4w_c_stock",
        a_freq_col:  "change_a_freq",
        b_freq_col:  "change_b_freq",
        c_freq_col:  "change_c_freq",
        a_ms_col:    "change_4w_a_ms",
        b_ms_col:    "change_4w_b_ms",
    }

    # 添加百分比变化列
    for col in metric_cols:
        out[rename_map[col]] = df_keep[f"__pct_{col}"].values

    # （可选）添加绝对变化列（便于做阈值/分段）
    if keep_delta:
        for col in metric_cols:
            out[rename_map[col].replace("change_", "delta_")] = df_keep[f"__delta_{col}"].values

    # ---------- 3) 计算 first_b_stock_rw（首次 B 份额>0 的 rolling_order） ----------
    # First rolling_order per account where b_ms_col > 0
    first_b = (
        volume_r4w.loc[volume_r4w[b_ms_col] > 0, [id_col, order_col]]
        .groupby(id_col, as_index=False)[order_col]
        .min()
        .rename(columns={order_col: "first_b_stock_rw"})
    )

    # 如果某账号从未出现 B 份额>0，则回退为该账号最后一次 rolling_order
    fallback = (
        volume_r4w.groupby(id_col, as_index=False)[order_col]
        .max()
        .rename(columns={order_col: "fallback_order"})
    )

    first_b = fallback.merge(first_b, on=id_col, how="left")
    first_b["first_b_stock_rw"] = first_b["first_b_stock_rw"].fillna(first_b["fallback_order"]).astype("Int32")

    out = out.merge(first_b[[id_col, "first_b_stock_rw"]], on=id_col, how="left")

    # ---------- 4) 清理临时列 ----------
    # （这里 out 已经是最终表，无需保留临时列）
    # 如果你希望保留更多上下文（如 curr/prev 原值），可以在 out 里额外 merge。

    return out


In [ ]:
change_r4w = get_change_r4w_fast(volume_r4w)

In [ ]:
# for loop version

def get_matrix(change_r4w, wolume_r4w):
    change_cols = ['change_4w_a_ms', 'change_4w_a_stock', 'change_a_freq']
    feature_records = []
    for acct, group in change_r4w.groupby('Account ID'):
        row={'Account ID': acct}
        first_b_stock_rw = group['first_b_stock_rw'].iloc[0]
        before = group[group['rolling_order'] < first_b_stock_rw]
        after = group[group['rolling_order'] >= first_b_stock_rw]
        for col in change_cols:
            row[f'{col}_neg_before'] = (before[col] < 0).sum()
            row[f'{col}_pos_before'] = (before[col] > 0).sum()
            row[f'{col}_no_change_before'] = (before[col] == 0).sum()
            row[f'{col}_neg_after'] = (after[col] < 0).sum()
            row[f'{col}_pos_after'] = (after[col] > 0).sum()
            row[f'{col}_no_change_after'] = (after[col] == 0).sum()
            row[f'{col}_volatility_before'] = before[col].std()
            row[f'{col}_volatility_after'] = after[col].std()
            row[f'{col}_first_drop_order'] = group.loc[group[col] < 0, 'rolling_order'].min() if not group.loc[group[col] < 0].empty else np.nan
            row[f'{col}_first_rise_order'] = group.loc[group[col] > 0, 'rolling_order'].min() if not group.loc[group[col] > 0].empty else np.nan
        feature_records.append(row)
    df_change_features = pd.DataFrame(feature_records)

    # b wine features (positive count, volatility)
    b_change_cols = ['change_4w_b_ms', 'change_4w_b_stock', 'change_b_freq']
    b_feature_records = []
    for acct, group in change_r4w.groupby('Account ID'):
        row={'Account ID': acct}
        for col in b_change_cols:
            row[f'{col}_pos_count'] = (group[col] > 0).sum()
            row[f'{col}_volatility'] = group[col].std()
        b_feature_records.append(row)
    df_b_change_summary = pd.DataFrame(b_feature_records)

    # consecutive_streaks
    def count_consecutive_streaks(series, condition, threshold):
        streak = 0
        count = 0
        for val in series:
            if pd.notnull(val) and condition(val):
                streak += 1
                if streak == threshold:
                    count += 1
                    streak -=1  # allow overlapping streaks
            else:
                streak = 0
        return count
    records = []
    for acct, group in change_r4w.groupby('Account ID'):
        row = {'Account ID': acct}
        first_b_order = group['first_b_stock_rw'].iloc[0]
        before = group[group['rolling_order'] < first_b_order]
        after = group[group['rolling_order'] >= first_b_order]
        row['a_stocking_3fall_before_b'] = count_consecutive_streaks(before['change_4w_a_stock'], lambda x: x < 0, 3)
        row['a_ms_3fall_after_b'] = count_consecutive_streaks(after['change_4w_a_ms'], lambda x: x < 0, 3)
        row['a_stocking_4fall_before_b'] = count_consecutive_streaks(before['change_4w_a_stock'], lambda x: x < 0, 4)
        row['a_ms_4fall_after_b'] = count_consecutive_streaks(after['change_4w_a_ms'], lambda x: x < 0, 4)
        row['a_stocking_5fall_before_b'] = count_consecutive_streaks(before['change_4w_a_stock'], lambda x: x < 0, 5)
        row['a_ms_5fall_after_b'] = count_consecutive_streaks(after['change_4w_a_ms'], lambda x: x < 0, 5)
        records.append(row)
    df_a_streaks = pd.DataFrame(records)

    #Merge all features + generate target
    df_features = df_change_features.merge(df_b_change_summary, on='Account ID', how='left')
    df_features = df_features.merge(df_a_streaks, on='Account ID', how='left')

    df_modeling = df_features.merge(df_all[['account_id', 'account_name']].drop_duplicates(),
                                  left_on='Account ID', right_on='account_id', how='left')
    return df_modeling


In [ ]:
import pandas as pd
import numpy as np

# =========================
# Scalling anbd Accelerated feature builder v2.1
# =========================
# Upgrads:
# - Features reproduction：neg/pos/zero, volatility, first_drop/rise
# - B Metrics：pos_count, volatility
# - A Metrics：before_B, a_stock consecutive decling 3/4/5；after_B, a_ms consecutive decling 3/4/5
# - Support cutoff（only keep <= cutoff rolling windows）
# - Includes before_B and after_B（as first_b_stock_rw cutoff）

def build_feature_matrix_fast(
    change_r4w: pd.DataFrame,
    volume_r4w: pd.DataFrame,
    cutoff: str | pd.Timestamp,
    *,
    id_col="Account ID",
    name_col_in_volume="account_name",
    order_col="rolling_order",
    firstb_col="first_b_stock_rw",
    #  change columns (a and b sides)
    a_change_cols=("change_4w_a_ms", "change_4w_a_stock", "change_a_freq"),
    b_change_cols=("change_4w_b_ms", "change_4w_b_stock", "change_b_freq"),
    # 
    # get end date from change_r4w (default parse from 'rolling_window_range_end' start date)
    end_range_col="rolling_window_range_end",
):
    """
    构建特征矩阵（含 before_B & after_B），只使用 <= cutoff 的窗口。
    Build feature matrix with before/after-B splits, using windows with end <= cutoff.
    """

    df = change_r4w.copy()

    # ---------- 0) 截止日期过滤（尽量不依赖额外列；从 range 字符串解析 end 日期） ----------
    # ---------- 0) Cutoff filtering (parse end date from range string to avoid extra merges) ----------
    # e.g. 'rolling_window_range_end' like '2025-01-03 to 2025-01-31' → 取前半段 '2025-01-03'
    # e.g. 'rolling_window_range_end' format: '2025-01-03 to 2025-01-31' → take first part '2025-01-03'
    if end_range_col in df.columns:
        end_dates = pd.to_datetime(df[end_range_col].astype(str).str.split(" to ").str[0], errors="coerce")
    else:
        # 后备：若无该列，可改为用 volume_r4w merge end 周五（需要确保两边可 join）
        # Fallback: if missing, consider merging end week from volume_r4w (ensure joinable)
        raise ValueError(f"Column '{end_range_col}' not found in change_r4w; please provide an end date column or adjust parsing.")

    cutoff = pd.to_datetime(cutoff)
    df = df.loc[end_dates.le(cutoff)].copy()

    # 必要列存在性校验
    # Necessary columns check
    need_cols = [id_col, order_col, firstb_col, *a_change_cols, *b_change_cols]
    missing = [c for c in need_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns in change_r4w: {missing}")

    # ---------- 1) 相位标记 phase flags ----------
    # ---------- 1) Phase flags ----------
    # before_B / after_B
    df["__before"] = df[order_col] < df[firstb_col]
    df["__after"]  = df[order_col] >= df[firstb_col]

    # ---------- 2) A 面：before/after 的 neg/pos/zero 计数 + volatility ----------
    # -----------2) A side: before/after neg/pos/zero counts + volatility ----------
    # 用掩码向量化：在不属于该相位的行置为 False / NaN，再 groupby sum/std
    # Use masks to vectorize: set non-phase rows to False/NaN, then groupby sum/std
    out_parts = []

    g = df.groupby(id_col, group_keys=True)

    def _phase_counts_and_vol(col: str, phase_col: str, phase_name: str) -> pd.DataFrame:
        m = df[phase_col].to_numpy()       # True/False
        vals = df[col].to_numpy()

        # counts
        neg_ct = g.apply(lambda x: ((x[col] < 0) & x[phase_col]).sum()).rename(f"{col}_neg_{phase_name}")
        pos_ct = g.apply(lambda x: ((x[col] > 0) & x[phase_col]).sum()).rename(f"{col}_pos_{phase_name}")
        zer_ct = g.apply(lambda x: ((x[col] == 0) & x[phase_col]).sum()).rename(f"{col}_no_change_{phase_name}")

        # volatility (std)
        # 用 where 只保留该相位的数据，其他相位设为 NaN，再按组 std
        # use where to keep only phase data, set others to NaN, then group std
        phase_vals = np.where(m, vals, np.nan)
        tmp = df[[id_col]].copy()
        tmp[f"__{col}_phase"] = phase_vals
        vol = tmp.groupby(id_col, as_index=True)[f"__{col}_phase"].std(ddof=1).rename(f"{col}_volatility_{phase_name}")

        return pd.concat([neg_ct, pos_ct, zer_ct, vol], axis=1)

    for col in a_change_cols:
        out_parts.append(_phase_counts_and_vol(col, "__before", "before"))
        out_parts.append(_phase_counts_and_vol(col, "__after",  "after"))

    df_change_features = pd.concat(out_parts, axis=1)
    df_change_features.index.name = id_col
    df_change_features = df_change_features.reset_index()

    # ---------- 3) first_drop / first_rise（基于全部窗口，不分相位） ----------
    # ---------- 3) first_drop / first_rise (across all windows, no phase split) ----------
    # 注意：这里按你的原逻辑，是在“所有窗口”上取 min(rolling_order)
    # Note: according to your original logic, we take min(rolling_order) across all windows
    fr_parts = []
    for col in (*a_change_cols, *b_change_cols):
        mask_drop = df[col] < 0
        mask_rise = df[col] > 0
        first_drop = df.loc[mask_drop].groupby(id_col)[order_col].min().rename(f"{col}_first_drop_order")
        first_rise = df.loc[mask_rise].groupby(id_col)[order_col].min().rename(f"{col}_first_rise_order")
        fr_parts.append(first_drop)
        fr_parts.append(first_rise)
    df_firsts = pd.concat(fr_parts, axis=1).reset_index()

    # ---------- 4) B 面汇总（不分相位）：pos_count & volatility ----------
    # ---------- 4) B side summary (no phase split): pos_count & volatility ----------
    b_parts = []
    for col in b_change_cols:
        pos_count = df.groupby(id_col)[col].apply(lambda s: (s > 0).sum()).rename(f"{col}_pos_count")
        vol = df.groupby(id_col)[col].std(ddof=1).rename(f"{col}_volatility")
        b_parts.append(pos_count)
        b_parts.append(vol)
    df_b_summary = pd.concat(b_parts, axis=1).reset_index()

    # ---------- 5) 连续串（小规模按组 apply，窗口≤52/较短 → 可接受） ----------
    # ---------- 5) Consecutive streaks (small-scale group apply, windows ≤52/short → acceptable) ----------
    # 允许重叠 / allow overlapping
    # 计算 k 连续负增长的次数 /count of k consecutive negative changes
    def _count_consecutive_neg(x: pd.Series, k: int) -> int:
        arr = x.to_numpy()
        n = 0
        streak = 0
        for v in arr:
            if pd.notna(v) and (v < 0):
                streak += 1
                if streak == k:
                    n += 1
                    streak -= 1  # allow overlapping
            else:
                streak = 0
        return n

    # A: before_B 的 a_stock 3/4/5 连跌
    # A: a_stocking_3/4/5 fall before_B
    # A: after_B  的 a_ms    3/4/5 连跌
    # A: a_ms_3/4/5 fall after_B
    # 先按相位切分再聚合
    # Then split by phase and aggregate
    def _streak_block(group: pd.DataFrame) -> pd.Series:
        out = {}
        g_before = group.loc[group["__before"]]
        g_after  = group.loc[group["__after"]]

        # before_B streaks on change_4w_a_stock
        for K in (3, 4, 5):
            out[f"a_stocking_{K}fall_before_b"] = _count_consecutive_neg(g_before["change_4w_a_stock"], K)

        # after_B streaks on change_4w_a_ms
        for K in (3, 4, 5):
            out[f"a_ms_{K}fall_after_b"] = _count_consecutive_neg(g_after["change_4w_a_ms"], K)

        return pd.Series(out)

    df_streaks = df.groupby(id_col, as_index=False).apply(_streak_block).reset_index().rename(columns={id_col: id_col})

    # ---------- 6) 合并所有特征 ----------
    # ---------- 6) Merge all features ----------
    features = (
        df_change_features
        .merge(df_firsts,    on=id_col, how="left")
        .merge(df_b_summary, on=id_col, how="left")
        .merge(df_streaks,   on=id_col, how="left")
    )

    # ---------- 7) 追加账户名称 ----------
    # ---------- 7) Append account names ----------
    if (name_col_in_volume in volume_r4w.columns) and (id_col in change_r4w.columns):
        names = volume_r4w[[name_col_in_volume]].copy()
        # volume_r4w 里通常有 'account_id'，但当前主键是 'Account ID'，统一一下映射：
        # volume_r4w usually has 'account_id', but current key is 'Account ID', unify mapping:
        # 尝试从 volume_r4w 找到 id 的列名
        # Try to find the id column name from volume_r4w
        id_candidates = [c for c in volume_r4w.columns if c.lower() in ("account id", "account_id", "id")]
        if id_candidates:
            id_in_volume = id_candidates[0]
            names = volume_r4w[[id_in_volume, name_col_in_volume]].drop_duplicates()
            names = names.rename(columns={id_in_volume: id_col})
            features = features.merge(names, on=id_col, how="left")

    return features


# =============================
# 便捷函数：一键生成 Train/Valid
# =============================
# ==========================================================
# Convenience function: build Train/Valid in one call
# ==========================================================
def build_train_valid_features(
    change_r4w: pd.DataFrame,
    volume_r4w: pd.DataFrame,
    *,
    cutoff_train="2025-08-29",
    cutoff_valid="2025-10-03",
    **kwargs
):
    """
    生成两套特征矩阵：
      - Train: 仅使用 <= cutoff_train 的窗口
      - Valid: 仅使用 <= cutoff_valid 的窗口（天然 out-of-fold 相对更近时段）
    """
    feat_train = build_feature_matrix_fast(change_r4w, volume_r4w, cutoff_train, **kwargs)
    feat_valid = build_feature_matrix_fast(change_r4w, volume_r4w, cutoff_valid, **kwargs)
    return feat_train, feat_valid


# ======================
#  How to use
# ======================
# feat_train, feat_valid = build_train_valid_features(
#     change_r4w,
#     volume_r4w,
#     cutoff_train="2025-08-29",
#     cutoff_valid="2025-10-03",
#     id_col="Account ID",
#     name_col_in_volume="account_name",
#     order_col="rolling_order",
#     firstb_col="first_b_stock_rw",
#     a_change_cols=("change_4w_a_ms", "change_4w_a_stock", "change_a_freq"),
#     b_change_cols=("change_4w_b_ms", "change_4w_b_stock", "change_b_freq"),
#     end_range_col="rolling_window_range_end",   # e.g. "2025-01-03 to 2025-01-31"
# )
#
# # 之后你就可以用 feat_train 做训练，feat_valid 做 out-of-fold 验证（再算 AUC/F1）。
# # Then you can use feat_train for training, and feat_valid for out-of-fold validation (e.g. compute AUC/F1).


In [ ]:
import pandas as pd
import numpy as np

# =========================
# Fast feature builder v3.0
# =========================
def build_feature_matrix_fast(
    change_r4w: pd.DataFrame,
    volume_r4w: pd.DataFrame,
    cutoff: str | pd.Timestamp,
    *,
    # —— 主键与列名 / Keys & column names ——
    id_col="Account ID",
    name_col_in_volume="account_name",
    order_col="rolling_order",
    firstb_col="first_b_stock_rw",
    # —— A/B 侧变化列 / change columns ——
    a_change_cols=("change_4w_a_ms", "change_4w_a_stock", "change_a_freq"),
    b_change_cols=("change_4w_b_ms", "change_4w_b_stock", "change_b_freq"),
    # —— 窗口终点列（用于截止过滤）/ window-end column for cutoff ——
    end_range_col="rolling_window_range_end",
    # —— before 段回看窗口长度 / lookback window length (before-B only) ——
    before_lookback_orders=52,
    # （可选）after 段限制长度；None=不限制
    post_weeks: int | None = None,
):
    """
    构建特征矩阵（含 before_B & after_B），且：
      - 仅使用窗口终点 <= cutoff 的行
      - before 段只统计 “first_B 之前最近 `before_lookback_orders` 个窗口”
      - 输出中强制包含 account_name（来自 volume_r4w）

    Build feature matrix with before/after-B splits, using windows with end <= cutoff.
    'before' phase only counts the last `before_lookback_orders` windows prior to first-B.
    Always returns `account_name` (merged from volume_r4w).
    """

    df = change_r4w.copy()

    # ---------- 0) 截止日期过滤（从 range 字符串解析 end 日期） ----------
    # Parse window end date from e.g. "2025-01-03 to 2025-01-31" -> take the first date
    if end_range_col not in df.columns:
        raise ValueError(
            f"Column '{end_range_col}' not found in change_r4w; "
            f"please provide a window-end column for cutoff filtering."
        )
    end_dates = pd.to_datetime(
        df[end_range_col].astype(str).str.split(" to ").str[0],
        errors="coerce"
    )
    cutoff = pd.to_datetime(cutoff)
    df = df.loc[end_dates.le(cutoff)].copy()

    # ---------- 1) 基本列检查 / sanity checks ----------
    need_cols = [id_col, order_col, firstb_col, *a_change_cols, *b_change_cols]
    missing = [c for c in need_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns in change_r4w: {missing}")

    # ---------- 2) 相位标记（before / after） ----------
    df = df.sort_values([id_col, order_col]).copy()
    df["__before"] = df[order_col] < df[firstb_col]
    df["__after"]  = df[order_col] >= df[firstb_col]

    # ---------- 3) 截断 before 段为“first_B 之前最近 52 个窗口” ----------
    # Limit 'before' phase to last `before_lookback_orders` windows prior to first-B
    bounds = (
        df[[id_col, firstb_col]]
        .drop_duplicates(subset=[id_col])
        .assign(__lb=lambda x: (x[firstb_col] - before_lookback_orders).clip(lower=1))
        .rename(columns={firstb_col: "__fb"})
    )
    df = df.merge(bounds[[id_col, "__lb", "__fb"]], on=id_col, how="left")
    mask_before_window = (df[order_col] >= df["__lb"]) & (df[order_col] < df["__fb"])
    df["__before"] = df["__before"] & mask_before_window

    # （可选）after 段限制长度（例如仅保留首购后 52 周）
    if post_weeks is not None:
        df["__after"] = df["__after"] & (df[order_col] < (df["__fb"] + post_weeks))

    # ---------- 4) A 面：before/after 的 neg/pos/zero + volatility ----------
    out_parts = []
    g = df.groupby(id_col, group_keys=True)

    def _phase_counts_and_vol(col: str, phase_col: str, phase_name: str) -> pd.DataFrame:
        # 用掩码保留该相位的数据，其余置 NaN；groupby.std 会自动忽略 NaN
        vals = df[col].to_numpy()
        m = df[phase_col].to_numpy()
        phase_vals = np.where(m, vals, np.nan)

        tmp = df[[id_col]].copy()
        tmp[f"__{col}_phase"] = phase_vals

        neg_ct = g.apply(lambda x: ((x[col] < 0) & x[phase_col]).sum()).rename(f"{col}_neg_{phase_name}")
        pos_ct = g.apply(lambda x: ((x[col] > 0) & x[phase_col]).sum()).rename(f"{col}_pos_{phase_name}")
        zer_ct = g.apply(lambda x: ((x[col] == 0) & x[phase_col]).sum()).rename(f"{col}_no_change_{phase_name}")
        vol    = tmp.groupby(id_col, as_index=True)[f"__{col}_phase"].std(ddof=1).rename(f"{col}_volatility_{phase_name}")

        return pd.concat([neg_ct, pos_ct, zer_ct, vol], axis=1)

    for col in a_change_cols:
        out_parts.append(_phase_counts_and_vol(col, "__before", "before"))
        out_parts.append(_phase_counts_and_vol(col, "__after",  "after"))

    df_change_features = pd.concat(out_parts, axis=1)
    df_change_features.index.name = id_col
    df_change_features = df_change_features.reset_index()

    # ---------- 5) first_drop / first_rise（基于“<=cutoff”的全部窗口） ----------
    fr_parts = []
    for col in (*a_change_cols, *b_change_cols):
        mask_drop = df[col] < 0
        mask_rise = df[col] > 0
        first_drop = df.loc[mask_drop].groupby(id_col)[order_col].min().rename(f"{col}_first_drop_order")
        first_rise = df.loc[mask_rise].groupby(id_col)[order_col].min().rename(f"{col}_first_rise_order")
        fr_parts.append(first_drop)
        fr_parts.append(first_rise)
    df_firsts = pd.concat(fr_parts, axis=1).reset_index()

    # ---------- 6) B 面汇总（不分相位）：pos_count & volatility ----------
    b_parts = []
    for col in b_change_cols:
        pos_count = df.groupby(id_col)[col].apply(lambda s: (s > 0).sum()).rename(f"{col}_pos_count")
        vol = df.groupby(id_col)[col].std(ddof=1).rename(f"{col}_volatility")
        b_parts.append(pos_count)
        b_parts.append(vol)
    df_b_summary = pd.concat(b_parts, axis=1).reset_index()

    # ---------- 7) 连续串（before: a_stock；after: a_ms） ----------
    def _count_consecutive_neg(x: pd.Series, k: int) -> int:
        arr = x.to_numpy()
        n, streak = 0, 0
        for v in arr:
            if pd.notna(v) and (v < 0):
                streak += 1
                if streak == k:
                    n += 1
                    streak -= 1  # 允许重叠 / allow overlapping
            else:
                streak = 0
        return n

    def _streak_block(group: pd.DataFrame) -> pd.Series:
        out = {}
        g_before = group.loc[group["__before"]]
        g_after  = group.loc[group["__after"]]
        for K in (3, 4, 5):
            out[f"a_stocking_{K}fall_before_b"] = _count_consecutive_neg(g_before["change_4w_a_stock"], K)
        for K in (3, 4, 5):
            out[f"a_ms_{K}fall_after_b"] = _count_consecutive_neg(g_after["change_4w_a_ms"], K)
        return pd.Series(out)

    df_streaks = df.groupby(id_col, as_index=False).apply(_streak_block).reset_index(drop=True)

    # ---------- 8) 合并所有特征 ----------
    features = (
        df_change_features
        .merge(df_firsts,    on=id_col, how="left")
        .merge(df_b_summary, on=id_col, how="left")
        .merge(df_streaks,   on=id_col, how="left")
    )

    # ---------- 9) 追加账户名，并将其置于第2列 ----------
    # Merge `account_name` from volume_r4w and move it right after id_col
    if name_col_in_volume in volume_r4w.columns:
        # 尝试识别 volume_r4w 的 ID 列名
        id_candidates = [c for c in volume_r4w.columns if c.lower() in ("account id", "account_id", "id")]
        if id_candidates:
            id_in_volume = id_candidates[0]
            names = volume_r4w[[id_in_volume, name_col_in_volume]].drop_duplicates()
            names = names.rename(columns={id_in_volume: id_col})
            features = features.merge(names, on=id_col, how="left")

    # 将列顺序调整为：id, account_name, 其它特征
    cols = features.columns.tolist()
    front = [id_col]
    if name_col_in_volume in cols:
        front.append(name_col_in_volume)
    rest = [c for c in cols if c not in front]
    features = features[front + rest]

    return features


# =============================
# 便捷函数：一键生成 Train/Valid
# =============================
def build_train_valid_features(
    change_r4w: pd.DataFrame,
    volume_r4w: pd.DataFrame,
    *,
    cutoff_train="2025-08-29",
    cutoff_valid="2025-10-03",
    **kwargs
):
    """
    生成两套特征矩阵：
      - Train: 仅使用 <= cutoff_train 的窗口
      - Valid: 仅使用 <= cutoff_valid 的窗口（更靠近当前，天然 out-of-fold）
    """
    feat_train = build_feature_matrix_fast(change_r4w, volume_r4w, cutoff_train, **kwargs)
    feat_valid = build_feature_matrix_fast(change_r4w, volume_r4w, cutoff_valid, **kwargs)
    return feat_train, feat_valid
